In [132]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def cstruct(send,results):
    global Q
    global Qs
    s_thread=send['Thread']
    s_index=send['Index']
    if s_index==1:
        return results
    temp=Q[(Q.Thread == s_thread) & (Q.Index==s_index-1)].iloc[0]
    if temp['Event']=='send':
        results.append(temp['ID'])
        send=temp
        return cstruct(send,results)
    else:
        results.append(temp['ID'])
        send=Qs.loc[temp.name]
        results.append(send['ID'])
        return cstruct(send,results)
    
def no_index(start,finish,t,event):
    #if type(event)==int:
    #    event=Qr[(Qr.ID==R[finish])].iloc[0]
    for l in range(start,finish):
        if t[l]>0 and R[l] in cstruct(event,results):
            return True
        else:
            continue
    return False

    
#============ここから定義============
    
heading_res=('Thread','Port','Index')
heading_snd=('Thread','Port','Event','Index')


#Q is SYN-sequence
Qr=pd.read_csv('SYN-res.csv',names=heading_res)
Qs=pd.read_csv('SYN-snd.csv',names=heading_snd)


r_list=list(range(1,len(Qr)+1))
r_list=list(map(lambda x:'r'+str(x),r_list)) #['r1', 'r2', 'r3', 'r4']
s_list=list(range(1,len(Qs)+1))
s_list=list(map(lambda x:'s'+str(x),s_list)) #['s1', 's2', 's3', 's4']

Qr.insert(0,'ID',r_list) #attach the name of event
Qs.insert(0,'ID',s_list) #attach the name of event

Q=[] #Q is SYN-sequence
Q=pd.merge(Qr,Qs,how='outer')
race_set={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]} #sample data



#============ここまで定義============

R=[] #create race_set
D=[] #number of race_set
heading=[] #{r1,r2,....r}

R.append('')
D.append('')
heading.append('dummy')
# set R,D
for key in race_set.keys():
    if len(race_set[key])>0:
        R.append(key) # raceのあるイベントを追加
        D.append(len(race_set[key])) #raceの数を追加
        heading.append(key) # r
table=pd.DataFrame([],columns=heading)

t=np.zeros(len(R)) #raceの数分の配列 #t[0]はダミー

while True:
    results=[]
    #num=0
    #num+=1　
    max_index=''
    for i in range(len(t)-1,0,-1):
        if t[i]<D[i] and t[i]!=-1:
            max_index=i
            break
    if max_index=='':
        break
    t[i]+=1
    
    if t[i]==1: #just changed t[i] from 0 to 1
        
        for j in tqdm(range(i+1,len(R)-1)):
            if t[j]!=-1 and Qr[Qr.ID=='r'+str(i)].iloc[0] in cstruct(Qr[Qr.ID=='r'+str(j)].iloc[0],[]):
                t[j]=-1
    
    for j in tqdm(range(i+1,len(R)-1)):
        if(t[j]==D[j]):
            t[j]=0 #just change t[j] from dj to 0
            
            for k in tqdm(range(j+1,len(R)-1)):
                if t[k]==-1 and Qr[Qr.ID=='r'+str(j)].iloc[0].ID in cstruct(Qr[Qr.ID=='r'+str(k)].iloc[0],[]) and no_index(0,k,t,Qr[(Qr.ID==R[k])].iloc[0]):
                    t[k]=0
    #let s be the t[i] sending event in race_set(ri)
    s= race_set[R[i]][int(t[i])-1] 

    
    if no_index(1,len(R)-1,t,Qs[(Qs.ID==s)].iloc[0])==False:
        table=table.append(pd.Series(t,index=table.columns),ignore_index=True)
        print(t[1:])
table.drop('dummy',axis=1)




0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

[0. 1.]
[1. 1.]


,r1,r3
0,0.0,1.0
1,1.0,1.0


In [128]:
table.drop('dummy',axis=1)

,r1,r3
0,0.0,1.0
1,1.0,1.0


In [92]:
no_index(1,len(R)-1,t,Qs[(Qs.ID==s)].iloc[0])==False

True

In [66]:
if R[1] in cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[1])].iloc[0],results):
    print(1)
else:
    print(0)

1

In [198]:
def cstruct(event,results):
    global Q
    global Qs

    thread=event['Thread']
    index=event['Index']
       
    if index==1:
        return ''
    for i in range(index-1,0,-1):
        temp=Q[(Q.Thread == thread) & (Q.Index==i)].iloc[0]
        if temp['Event']=='send':
            results.append(temp['ID'])
            event=temp
            results.extend(cstruct(event,results))
        else:
            results.append(temp['ID'])
            send=Qs.loc[temp.name]
            results.append(send['ID'])
            results.extend(cstruct(send,results))
    return listset(results))

In [188]:
sample=[1,2,4]
sample.extend(sample)
sample

[1, 2, 4, 1, 2, 4]

In [199]:
for i in range(0,4):
    print('r'+str(i+1),cstruct(Qr.iloc[i],[]))
    print('s'+str(i+1),cstruct(Qs.iloc[i],[]))

r1 
s1 
r2 ['r1', 's1']
s2 
r3 
s3 ['r2', 's2', 'r1', 's1']
r4 ['r3', 's3', 'r2', 's2', 'r1', 's1', 'r3', 's3', 'r2', 's2', 'r1', 's1']
s4 ['s2']


In [177]:
cstruct(Qr.iloc[3],[])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satoshi Fujita\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-177-0f44a770cc9e>", line 1, in <module>
    cstruct(Qr.iloc[3],[])
  File "<ipython-input-176-52fc8b02edb9>", line 20, in cstruct
    results.append(cstruct(event,results))
  File "<ipython-input-176-52fc8b02edb9>", line 20, in cstruct
    results.append(cstruct(event,results))
  File "<ipython-input-176-52fc8b02edb9>", line 20, in cstruct
    results.append(cstruct(event,results))
  [Previous line repeated 1541 more times]
  File "<ipython-input-176-52fc8b02edb9>", line 10, in cstruct
    print(i)
  File "C:\Users\Satoshi Fujita\anaconda3\lib\site-packages\ipykernel\iostream.py", line 404, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "C:\Users\Satoshi Fujita\anaconda3\lib\site-packages\ipykernel\iostream.py", line 205, in 

KeyboardInterrupt: 

In [183]:
Q

,ID,Thread,Port,Index,Event
0,r1,T2,{p2},1,NaN
1,r2,T2,{p2},2,NaN
2,r3,T3,{p3},1,NaN
3,r4,T3,{p3},2,NaN
4,s1,T1,{p2},1,send
5,s2,T4,{p2},1,send
6,s3,T2,{p3},3,send
7,s4,T4,{p3},2,send


In [114]:
i

0

In [51]:
event=0
if type(event)==int:
    print(1)

1


In [14]:
Qr.iloc[0+1]

ID          r1
Thread      T1
Port      {p2}
Index        1
Name: 0, dtype: object

In [240]:
Qs.loc[temp.name]

NameError: name 'temp' is not defined